In [ ]:
import boto3
account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

In [ ]:
import sagemaker
from sagemaker.estimator import Estimator

In [ ]:
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

In [ ]:
images_s3uri = 's3://{0}/stable-diffusion-webui/images/'.format(bucket)

In [ ]:
!aws s3 cp images $images_s3uri --recursive

In [ ]:
"""
model_name: str = "",
adam_beta1: float = 0.9,
adam_beta2: float = 0.999,
adam_epsilon: float = 1e-8,
adam_weight_decay: float = 0.01,
attention: str = "default",
center_crop: bool = True,
concepts_path: str = "",
custom_model_name: str = "",
epoch_pause_frequency: int = 0,
epoch_pause_time: int = 0,
gradient_accumulation_steps: int = 1,
gradient_checkpointing: bool = True,
half_model: bool = False,
has_ema: bool = False,
hflip: bool = False,
learning_rate: float = 0.00000172,
lora_learning_rate: float = 1e-4,
lora_txt_learning_rate: float = 5e-5,
lr_scheduler: str = 'constant',
lr_warmup_steps: int = 0,
max_token_length: int = 75,
max_train_steps: int = 1000,
mixed_precision: str = "fp16",
model_path: str = "",
not_cache_latents=False,
num_train_epochs: int = 1,
pad_tokens: bool = True,
pretrained_vae_name_or_path: str = "",
prior_loss_weight: float = 1.0,
resolution: int = 512,
revision: int = 0,
sample_batch_size: int = 1,
save_class_txt: bool = False,
save_embedding_every: int = 500,
save_preview_every: int = 500,
save_use_global_counts: bool = False,
save_use_epochs: bool = False,
scale_lr: bool = False,
scheduler: str = "ddim",
src: str = "",
shuffle_tags: bool = False,
train_batch_size: int = 1,
train_text_encoder: bool = True,
use_8bit_adam: bool = True,
use_concepts: bool = False,
use_cpu: bool = False,
use_ema: bool = True,
use_lora: bool = False,
v2: bool = False,
c1_class_data_dir: str = "",
c1_class_guidance_scale: float = 7.5,
c1_class_infer_steps: int = 60,
c1_class_negative_prompt: str = "",
c1_class_prompt: str = "",
c1_class_token: str = "",
c1_instance_data_dir: str = "",
c1_instance_prompt: str = "",
c1_instance_token: str = "",
c1_max_steps: int = -1,
c1_n_save_sample: int = 1,
c1_num_class_images: int = 0,
c1_sample_seed: int = -1,
c1_save_guidance_scale: float = 7.5,
c1_save_infer_steps: int = 60,
c1_save_sample_negative_prompt: str = "",
c1_save_sample_prompt: str = "",
c1_save_sample_template: str = "",
c2_class_data_dir: str = "",
c2_class_guidance_scale: float = 7.5,
c2_class_infer_steps: int = 60,
c2_class_negative_prompt: str = "",
c2_class_prompt: str = "",
c2_class_token: str = "",
c2_instance_data_dir: str = "",
c2_instance_prompt: str = "",
c2_instance_token: str = "",
c2_max_steps: int = -1,
c2_n_save_sample: int = 1,
c2_num_class_images: int = 0,
c2_sample_seed: int = -1,
c2_save_guidance_scale: float = 7.5,
c2_save_infer_steps: int = 60,
c2_save_sample_negative_prompt: str = "",
c2_save_sample_prompt: str = "",
c2_save_sample_template: str = "",
c3_class_data_dir: str = "",
c3_class_guidance_scale: float = 7.5,
c3_class_infer_steps: int = 60,
c3_class_negative_prompt: str = "",
c3_class_prompt: str = "",
c3_class_token: str = "",
c3_instance_data_dir: str = "",
c3_instance_prompt: str = "",
c3_instance_token: str = "",
c3_max_steps: int = -1,
c3_n_save_sample: int = 1,
c3_num_class_images: int = 0,
c3_sample_seed: int = -1,
c3_save_guidance_scale: float = 7.5,
c3_save_infer_steps: int = 60,
c3_save_sample_negative_prompt: str = "",
c3_save_sample_prompt: str = "",
c3_save_sample_template: str = "",
concepts_list=None
"""

In [ ]:
dreambooth_params = [
  "",
  0.9,
  0.999,
  1e-08,
  0.01,
  "default",
  False,
  "",
  "",
  0.0,
  60.0,
  1,
  True,
  False,
  "",
  True,
  2e-06,
  0.0002,
  0.0002,
  "constant",
  500,
  75,
  0,
  "no",
  "",
  True,
  100,
  True,
  "",
  1,
  512,
  "",
  1,
  True,
  500,
  500,
  True,
  False,
  False,
  "",
  "",
  False,
  1,
  True,
  False,
  False,
  False,
  False,
  False,
  "",
  "",
  7.5,
  40,
  "",
  "",
  "photo of dog",
  "/opt/ml/input/data/concepts/images",
  "",
  "photo of awsdogtoy dog",
  -1,
  1,
  0,
  -1,
  7.5,
  40,
  "",
  "",
  "",
  "",
  7.5,
  40,
  "",
  "",
  "",
  "",
  "",
  "",
  -1,
  1,
  0,
  -1,
  7.5,
  40,
  "",
  "",
  "",
  "",
  7.5,
  40,
  "",
  "",
  "",
  "",
  "",
  "",
  -1,
  1,
  0,
  -1,
  7.5,
  40,
  "",
  "",
  ""
]

In [ ]:
import uuid, json
dreambooth_config_id = str(uuid.uuid4())
dreambooth_config_file =f'{dreambooth_config_id}.json'
json.dump(dreambooth_params, open(dreambooth_config_file,'w'), indent=6)

In [ ]:
image_uri = '{0}.dkr.ecr.{1}.amazonaws.com/all-in-one-ai-stable-diffusion-webui-training'.format(account_id, region_name)
models_s3uri = 's3://{0}/stable-diffusion-webui/models/'.format(bucket)
dreambooth_s3uri = 's3://{0}/stable-diffusion-webui/dreambooth/'.format(bucket)
dreambooth_config_s3uri = 's3://{0}/stable-diffusion-webui/dreambooth-config/'.format(bucket)
config_s3uri = 's3://{0}/stable-diffusion-webui/dreambooth-config/'.format(bucket)

In [ ]:
print(dreambooth_config_file)
print(dreambooth_config_s3uri)
!aws s3 cp $dreambooth_config_file $dreambooth_config_s3uri

In [ ]:
def json_encode_hyperparameters(hyperparameters):
    for (k, v) in hyperparameters.items():
        print(k, v)
    
    return {str(k): json.dumps(v) for (k, v) in hyperparameters.items()}

train_args = {
    'train_dreambooth_settings': {
        'db_create_new_db_model': True, 
        'db_new_model_name': 'aws-dog-toy-model', 
        'db_new_model_src': '768-v-ema.ckpt', 
        'db_new_model_scheduler': 'ddim', 
        'db_create_from_hub': False, 
        'db_new_model_url': '', 
        'db_new_model_token': '', 
        'db_new_model_extract_ema': False, 
        'db_model_name': '', 
        'db_lora_model_name': '', 
        'db_lora_weight': 1, 
        'db_lora_txt_weight': 1, 
        'db_train_imagic_only': False, 
        'db_use_subdir': False, 
        'db_custom_model_name': '', 
        'db_train_wizard_person': False, 
        'db_train_wizard_object': True, 
        'db_performance_wizard': True
    }
}


hyperparameters = {
    'train-args': json.dumps(train_args),
    'train-task': 'dreambooth',
    'sd-models-s3uri': models_s3uri,
    'db-models-s3uri': dreambooth_s3uri,
    'dreambooth-config-id': dreambooth_config_id,
    'api-endpoint': 'noapi'
}

hyperparameters = json_encode_hyperparameters(hyperparameters)

instance_type = 'ml.g4dn.2xlarge'

In [ ]:
inputs = {
    'images': images_s3uri,
    'models': models_s3uri,
    'config': config_s3uri
}

In [ ]:
# Please exectute tools/prepare.py with the path to your model files directory.
# Make sure the 768-v-ema.ckpt and 768-v-ema.yaml have been uploaded to sd_models_s3uri

In [ ]:
estimator = Estimator(
    role = role,
    instance_count=1,
    instance_type = instance_type,
    image_uri = image_uri,
    hyperparameters = hyperparameters
)

In [ ]:
estimator.fit(inputs)